In [1]:
#For Google Collab
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


Imports that are needed

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [4]:
from google.colab import files
uploaded = files.upload()

Saving heart_train.csv to heart_train.csv


In [0]:
#!rm "heart_train.csv"
#!rm "heart_test.csv"

In [0]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [0]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]
CSV_COLUMNS = ['sbp', 'tobacco', 'ldl', 'adiposity', 'famhist', 'typea', 'obesity', 'alcohol', 'age', 'chd']

In [7]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=1, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(['heart_train.csv'], select_columns=CSV_COLUMNS)
raw_test_data = get_dataset(['heart_test.csv'], select_columns=CSV_COLUMNS)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [9]:
show_batch(raw_train_data)

sbp                 : [168]
tobacco             : [11.4]
ldl                 : [5.08]
adiposity           : [26.66]
famhist             : [b'Present']
typea               : [56]
obesity             : [27.04]
alcohol             : [2.61]
age                 : [59]


Data Preprocessing

Continuous Data

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [11]:
NUMERIC_FEATURES = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age']
packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

show_batch(packed_train_data)

famhist             : [b'Present']
numeric             : [[136.     3.15   4.37  20.22  59.    25.12  47.16  31.  ]]


Data Normalization

In [12]:
import pandas as pd
desc = pd.read_csv('heart_train.csv')[NUMERIC_FEATURES].describe()
desc

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age
count,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000
mean,137.219697,3.641591,4.773813,25.392247,53.477273,26.137374,16.792121,43.138889
std,18.965784,4.483097,2.022398,7.666093,9.896272,4.225893,24.407237,14.413369
min,101.000000,0.000000,0.980000,7.120000,13.000000,17.750000,0.000000,15.000000
25%,124.000000,0.097500,3.277500,19.740000,47.000000,23.200000,0.510000,31.750000
50%,134.000000,2.010000,4.400000,26.150000,54.000000,25.830000,7.710000,45.000000
75%,148.000000,5.500000,5.900000,31.057500,60.000000,28.430000,23.225000,56.000000
max,208.000000,31.200000,15.330000,42.490000,78.000000,46.580000,147.190000,64.000000


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [0]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)

Categorical Data

In [0]:
CATEGORIES = {
    'famhist': ['Present', 'Absent']
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

# See what you just created.
categorical_columns

categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)

Combined preprocessing layer

In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

Build the model

Initial Model

In [0]:
#model = tf.keras.Sequential([
  #preprocessing_layer,
  #tf.keras.layers.Dense(128, activation='relu'),
  #tf.keras.layers.Dense(128, activation='relu'),
  #tf.keras.layers.Dense(1, activation='sigmoid'),
#])

#model.compile(
  #loss='binary_crossentropy',
  #optimizer='adam',
  #metrics=['accuracy'])

Improved Model by adding regularization and droupout

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

Train and Evaluate the model

In [23]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

model.fit(train_data, epochs=50, validation_data=test_data)


Epoch 1/50
396/396 [==============================] - 3s 9ms/step - loss: 0.6079 - accuracy: 0.7071 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
396/396 [==============================] - 1s 3ms/step - loss: 0.5851 - accuracy: 0.7121 - val_loss: 0.5061 - val_accuracy: 0.7273
Epoch 3/50
396/396 [==============================] - 1s 3ms/step - loss: 0.5658 - accuracy: 0.7247 - val_loss: 0.4970 - val_accuracy: 0.7727
Epoch 4/50
396/396 [==============================] - 1s 2ms/step - loss: 0.5361 - accuracy: 0.7121 - val_loss: 0.5368 - val_accuracy: 0.7273
Epoch 5/50
396/396 [==============================] - 1s 3ms/step - loss: 0.5346 - accuracy: 0.7525 - val_loss: 0.5248 - val_accuracy: 0.7424
Epoch 6/50
396/396 [==============================] - 1s 2ms/step - loss: 0.5284 - accuracy: 0.7475 - val_loss: 0.5405 - val_accuracy: 0.7424
Epoch 7/50
396/396 [==============================] - 1s 2ms/step - loss: 0.5332 - accuracy: 0.7525 - val_loss: 0.5225 - val_accuracy: 0.757

In [24]:
print("--Evaluate model--")
model_loss, model_acc = model.evaluate(test_data, verbose=2)
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuray: {model_acc*100:.1f}%")

--Evaluate model--
66/66 - 0s - loss: 0.6062 - accuracy: 0.7879
Model Loss:    0.61
Model Accuray: 78.8%
